In [ ]:
import pandas
import numpy

from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import LSTM
from keras.models import Sequential

pandas.set_option('display.max_rows', 10)
pandas.set_option('display.max_columns', 10)

# set a fixed seed for numpy pseudo random generator
numpy.random.seed(100)

In [ ]:
data = pandas.read_csv("./datasets/gold_data_20170407to20170810.csv", parse_dates=['date'])

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
target = 'iran_gold.close'

In [ ]:
train_percent = 0.8
lag_step = 1
margin = int(train_percent*data.shape[0])
new_data = pandas.DataFrame({"x": [i for i in data[target]],
                             "y": [i for i in data[target].shift(lag_step)]})
new_data.dropna(inplace=True)
train = new_data.iloc[0:margin]
test = new_data.iloc[margin:]

In [ ]:
train.head()
test.head()

In [ ]:
def load_data(pdf, mem=100):
    
    docX, docY = [], []
    for i in range(len(pdf)-mem):
        docX.append(pdf.iloc[i:i+mem].as_matrix())
        docY.append(pdf.iloc[i+mem].as_matrix())
        
    return numpy.array(docX), numpy.array(docY)

X_train, Y_train = load_data(train[0:1000])
X_test, Y_test = load_data(test[0:1000])
Y_train

In [ ]:
in_out_neurons = (None, 2)
hidden_neurons = 100

model = Sequential()
model.add(LSTM(hidden_neurons, input_shape=in_out_neurons, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(in_out_neurons[1], input_dim=hidden_neurons))
model.add(Activation("linear"))
model.compile(loss="mean_squared_error", optimizer="rmsprop")
model.fit(X_train, Y_train, batch_size=500, epochs=10, validation_split=0.1)

pred = model.predict(X_test)
y_hat = pandas.DataFrame(pred, columns=['t-1', 't'])
y = pandas.DataFrame(Y_test, columns=['t-1', 't'])

In [ ]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file

In [ ]:
p1 = figure(x_axis_type="datetime", title="Gold High Prices")
p1.grid.grid_line_alpha = 0.3
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Price'

p1.line(y_hat.index, y_hat['t'], color='#A6CEE3', legend='y_hat')
p1.line(y.index, y['t'], color='#B2DF8A', legend='y')
p1.legend.location = "top_left"

output_file("/tmp/index.html", title="Iran Gold[High] Price Prediction")

show(gridplot([[p1]], plot_width=600, plot_height=600, sizing_mode='stretch_both'))